In [42]:
import normals_lib
import kaolin as kal
import torch
import numpy as np
import meshplot as mp

#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj") # Modelo de vaca
#mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
mesh = kal.rep.TriangleMesh.from_obj("pear.obj")

In [43]:
normals = normals_lib.compute_simple_vertex_normals(mesh).to(device=torch.device('cpu'))
normals_lib.visualize_normals(mesh, normals)
normals = normals.to(device=torch.device('cuda'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.9421380…

In [44]:
# Calculate principal curvatures and curvature directions
import curvature

k1, k2, e1, e2 = curvature.compute_curvatures(mesh)

In [45]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2))

In [38]:
import importlib
importlib.reload(dcurv)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [6]:
DwKr, kr = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc)

In [47]:
tridc = torch.zeros(dc.shape)
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])
DwKrtri, krtri = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=tridc.to(device=torch.device('cuda')))

In [49]:
colour = torch.zeros(mesh.vertices.shape[0], 3)
#colour[:,0] = DwKr.to(device=torch.device('cpu'))
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKrtri.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet"})
mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=krtri.to(device=torch.device('cpu')).numpy(),
        shading={'colormap': "jet"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.9421380…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.9421380…